In [1]:
# Autoreload imports
%load_ext autoreload
%autoreload 2

# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import plotly.express as px

from ecodynelec.parameter import Parameter  # Import the class to manipulate parameters
from ecodynelec.preprocessing.downloading import download #import the class to download entsoe data
from examples.mix_analysis.analysis_functions import format_data_0, get_metrics, plot_hourly_heatmap, plot_years, \
    plot_typical_days

# change plt font size
plt.rcParams.update({'font.size': 12})

In [2]:
# EcoDynElec configuration
my_config = Parameter()
my_config.freq = "H"
my_config.ctry = ['AT', 'CH', 'DE', 'FR', 'IT']
#my_config.target = ['CH']  # You can put multiple targets here
# Enable this if you want to plot fig 6 (will impact the computation time - it took me 15min)
my_config.target = ['AT', 'CH', 'DE', 'FR', 'IT']
my_config.data_cleaning = True
my_config.sg_imports = True

# Indicate where to save generation data
my_config.path.generation = "./test_data/downloads/generations" 
# Indicate where to save exchange data
my_config.path.exchanges = "./test_data/downloads/exchanges/"
# Recommended config: include swiss grid residual (difference between swiss grid and entso-e data)
my_config.residual_global = True
# Recommended config: include real (2020-2022) and estimated (2016-2019) renewable production (using EcoDynElec-Enr-Model)
my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2024_completed.csv'
#my_config.ch_enr_model_path = '../support_files/enr_prod_2016-2023_completed.csv'
# Years to study
years = ['2016','2017','2018','2019','2020','2021','2022','2023','2024']

In [3]:
from examples.mix_analysis.analysis_functions import load_data

flows, prods, mixs, prod_impacts, impacts = load_data(my_config.target, my_config.freq, years,
                                                      savedir="./mix_analysis/results_local_all_enr/")

In [4]:
prod_impacts

{'2016': {'AT': {'At plant':                      Residual_Hydro_Water_Reservoir_AT  \
   2016-01-01 00:00:00                                  0   
   2016-01-01 01:00:00                                  0   
   2016-01-01 02:00:00                                  0   
   2016-01-01 03:00:00                                  0   
   2016-01-01 04:00:00                                  0   
   ...                                                ...   
   2016-12-31 19:00:00                                  0   
   2016-12-31 20:00:00                                  0   
   2016-12-31 21:00:00                                  0   
   2016-12-31 22:00:00                                  0   
   2016-12-31 23:00:00                                  0   
   
                        Residual_Hydro_Run-of-river_and_poundage_AT  \
   2016-01-01 00:00:00                                            0   
   2016-01-01 01:00:00                                            0   
   2016-01-01 02:00:00   

In [5]:
flows_CH = [flows[i]['CH'] for i in flows.keys()]
flows_CH=pd.concat(flows_CH)


flows_AT = [flows[i]['AT'] for i in flows.keys()]
flows_AT=pd.concat(flows_AT)


flows_DE = [flows[i]['DE'] for i in flows.keys()]
flows_DE=pd.concat(flows_DE)


flows_FR = [flows[i]['FR'] for i in flows.keys()]
flows_FR=pd.concat(flows_FR)


flows_IT = [flows[i]['IT'] for i in flows.keys()]
flows_IT=pd.concat(flows_IT)

In [6]:
flows_IT

,production,imports,exports
2016-01-01 00:00:00,19300.0,3600.0,168.000
2016-01-01 01:00:00,18100.0,3250.0,151.000
2016-01-01 02:00:00,17100.0,3260.0,93.300
2016-01-01 03:00:00,16300.0,3480.0,114.000
2016-01-01 04:00:00,16200.0,3630.0,115.000
...,...,...,...
2024-12-31 19:00:00,24800.0,5210.0,14.600
2024-12-31 20:00:00,21800.0,5570.0,16.200
2024-12-31 21:00:00,19300.0,6120.0,0.226
2024-12-31 22:00:00,17900.0,6610.0,15.500


In [7]:
prods['2016']['AT'].sum(axis=1)

2016-01-01 00:00:00    1.000815
2016-01-01 01:00:00    1.000625
2016-01-01 02:00:00    1.000536
2016-01-01 03:00:00    1.000647
2016-01-01 04:00:00    0.999517
                         ...   
2016-12-31 19:00:00    1.000229
2016-12-31 20:00:00    1.000332
2016-12-31 21:00:00    1.000572
2016-12-31 22:00:00    0.999783
2016-12-31 23:00:00    1.000254
Length: 8784, dtype: float64

In [8]:
from examples.mix_analysis.analysis_functions import concatenate_and_format_data

# NOTE: Unless you use multiple targets (see section 0.), these dicts will contain only one element, corresponding to the target country
raw_productions_by_src, raw_consumptions_by_src, electricity_prod_mixs, electricity_mixs, producing_electricity_impacts, electricity_impacts = concatenate_and_format_data(my_config.target, years, flows, prods, mixs, prod_impacts, impacts)

In [9]:
electricity_impact_by_src_IT = {}
electricity_impact_by_src_AT = {}
electricity_impact_by_src_CH = {}
electricity_impact_by_src_FR = {}
electricity_impact_by_src_DE = {}
for voltage in ['At plant', 'High Voltage','Medium Voltage','Low Voltage']:
    # Sauvegarder les impacts d'électricité pour l'Italie (déjà existant)
    electricity_impact_by_src_IT[voltage] = electricity_impacts['IT'][voltage]['raw_df']


    # Sauvegarder les impacts d'électricité pour l'Autriche (AT)
    electricity_impact_by_src_AT[voltage] = electricity_impacts['AT'][voltage]['raw_df']


    # Sauvegarder les impacts d'électricité pour la Suisse (CH)
    electricity_impact_by_src_CH[voltage] = electricity_impacts['CH'][voltage]['raw_df']


    # Sauvegarder les impacts d'électricité pour la France (FR)
    electricity_impact_by_src_FR[voltage] = electricity_impacts['FR'][voltage]['raw_df']


    # Sauvegarder les impacts d'électricité pour l'Allemagne (DE)
    electricity_impact_by_src_DE[voltage] = electricity_impacts['DE'][voltage]['raw_df']

In [10]:
electricity_impact_by_src_DE

{'At plant':                      Mix_Other  Residual_Hydro_Water_Reservoir_AT  \
 2016-01-01 00:00:00       1.14                                  0   
 2016-01-01 01:00:00       1.16                                  0   
 2016-01-01 02:00:00       1.18                                  0   
 2016-01-01 03:00:00       1.07                                  0   
 2016-01-01 04:00:00       1.08                                  0   
 ...                        ...                                ...   
 2024-12-31 19:00:00       1.66                                  0   
 2024-12-31 20:00:00       1.95                                  0   
 2024-12-31 21:00:00       2.08                                  0   
 2024-12-31 22:00:00       2.19                                  0   
 2024-12-31 23:00:00       2.09                                  0   
 
                      Residual_Hydro_Run-of-river_and_poundage_AT  \
 2016-01-01 00:00:00                                            0   
 2016-01

In [11]:
tot_consumption_CH = raw_consumptions_by_src['CH']['df']


# Sauvegarder la consommation totale pour l'Autriche (AT)
tot_consumption_AT = raw_consumptions_by_src['AT']['df']


# Sauvegarder la consommation totale pour la France (FR)
tot_consumption_FR = raw_consumptions_by_src['FR']['df']

# Sauvegarder la consommation totale pour l'Allemagne (DE)
tot_consumption_DE = raw_consumptions_by_src['DE']['df']


# Sauvegarder la consommation totale pour l'Italie (IT)
tot_consumption_IT = raw_consumptions_by_src['IT']['df']

In [12]:
tot_consumption_IT

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,58.756858,1015.706814,705.656201,1329.274159,19158.615982,447.820400,22715.830412
2016-01-01 01:00:00,56.433173,927.751255,680.218736,1073.595796,17985.422391,474.857600,21198.278952
2016-01-01 02:00:00,72.968105,1010.162653,782.978621,779.890989,17029.945876,589.760970,20265.707215
2016-01-01 03:00:00,59.180510,1008.546306,685.684963,1203.061650,16203.407380,493.616600,19653.497409
2016-01-01 04:00:00,56.183336,1035.183371,660.855277,1365.776340,16126.475700,475.131500,19719.605525
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,288.676809,1992.243476,1037.419405,1481.094864,24796.867231,392.939740,29989.241524
2024-12-31 20:00:00,338.119720,1717.639473,1218.725308,1595.361148,21784.757797,697.521900,27352.125346
2024-12-31 21:00:00,343.052331,1829.017814,1310.452645,1732.459277,19302.378288,910.027909,25427.388265
2024-12-31 22:00:00,255.210057,2050.558292,1358.869864,1972.909502,17894.187536,962.633850,24494.369101


In [13]:
raw_consumption_by_src_IT = raw_consumptions_by_src['IT']['raw_df']


# Sauvegarder la consommation brute pour l'Autriche (AT)
raw_consumption_by_src_AT = raw_consumptions_by_src['AT']['raw_df']


# Sauvegarder la consommation brute pour la Suisse (CH)
raw_consumption_by_src_CH = raw_consumptions_by_src['CH']['raw_df']


# Sauvegarder la consommation brute pour la France (FR)
raw_consumption_by_src_FR = raw_consumptions_by_src['FR']['raw_df']


# Sauvegarder la consommation brute pour l'Allemagne (DE)
raw_consumption_by_src_DE = raw_consumptions_by_src['DE']['raw_df']

In [14]:
raw_consumption_by_src_CH

,Mix_Other,Residual_Hydro_Water_Reservoir_AT,Residual_Hydro_Run-of-river_and_poundage_AT,Residual_Other_AT,Biomass_AT,Fossil_Brown_coal/Lignite_AT,Fossil_Coal-derived_gas_AT,Fossil_Gas_AT,Fossil_Hard_coal_AT,Fossil_Oil_AT,...,Solar_IT,Waste_IT,Wind_Offshore_IT,Wind_Onshore_IT,Hydro_Pumpage_IT,Energy_storage_AT,Energy_storage_CH,Energy_storage_DE,Energy_storage_FR,Energy_storage_IT
2016-01-01 00:00:00,217.543033,0.0,0.0,0.0,21.200054,0.0,0.0,50.228885,10.530746,0.0,...,0.004025,0.184981,0.0,0.879872,0.0,NaN,NaN,NaN,NaN,NaN
2016-01-01 01:00:00,231.163818,0.0,0.0,0.0,22.343494,0.0,0.0,52.556394,11.171747,0.0,...,0.000000,0.148957,0.0,0.906387,0.0,NaN,NaN,NaN,NaN,NaN
2016-01-01 02:00:00,263.611761,0.0,0.0,0.0,25.738471,0.0,0.0,57.703992,13.007614,0.0,...,0.000000,0.074033,0.0,0.432434,0.0,NaN,NaN,NaN,NaN,NaN
2016-01-01 03:00:00,212.952282,0.0,0.0,0.0,21.762522,0.0,0.0,46.662585,11.014773,0.0,...,0.000000,0.087451,0.0,0.421899,0.0,NaN,NaN,NaN,NaN,NaN
2016-01-01 04:00:00,194.815195,0.0,0.0,0.0,20.409211,0.0,0.0,42.872596,10.337133,0.0,...,0.000000,0.080842,0.0,0.441316,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,244.907234,0.0,0.0,0.0,18.061909,0.0,0.0,114.034931,0.000000,0.0,...,0.000000,0.098728,0.0,0.088779,0.0,0.0,0.0,0.0,0.0,0.000000
2024-12-31 20:00:00,329.473559,0.0,0.0,0.0,23.293014,0.0,0.0,135.620511,0.000000,0.0,...,0.000000,0.125660,0.0,0.120296,0.0,0.0,0.0,0.0,0.0,0.000000
2024-12-31 21:00:00,345.294317,0.0,0.0,0.0,23.019621,0.0,0.0,128.230104,0.000000,0.0,...,0.000000,0.001947,0.0,0.001947,0.0,0.0,0.0,0.0,0.0,0.000000
2024-12-31 22:00:00,436.560405,0.0,0.0,0.0,26.704997,0.0,0.0,147.729768,0.000000,0.0,...,0.000000,0.142995,0.0,0.159094,0.0,0.0,0.0,0.0,0.0,0.000000


In [15]:
# Sauvegarder la production totale pour l'Italie (IT) - déjà existant
tot_production_IT = raw_productions_by_src['IT']['df']


# Sauvegarder la production totale pour l'Autriche (AT)
tot_production_AT = raw_productions_by_src['AT']['df']


# Sauvegarder la production totale pour la Suisse (CH)
tot_production_CH = raw_productions_by_src['CH']['df']


# Sauvegarder la production totale pour la France (FR)
tot_production_FR = raw_productions_by_src['FR']['df']


# Sauvegarder la production totale pour l'Allemagne (DE)
tot_production_DE = raw_productions_by_src['DE']['df']

In [16]:
tot_production_DE

,DE,sum
2016-01-01 00:00:00,48210.987190,48210.987190
2016-01-01 01:00:00,47824.776174,47824.776174
2016-01-01 02:00:00,47553.802296,47553.802296
2016-01-01 03:00:00,47443.194672,47443.194672
2016-01-01 04:00:00,46306.613955,46306.613955
...,...,...
2024-12-31 19:00:00,57196.911200,57196.911200
2024-12-31 20:00:00,54762.516800,54762.516800
2024-12-31 21:00:00,53590.191200,53590.191200
2024-12-31 22:00:00,53512.786500,53512.786500


In [17]:
production_IT = raw_productions_by_src['IT']['raw_df']


# Sauvegarder la production brute pour l'Autriche (AT)
production_AT = raw_productions_by_src['AT']['raw_df']


# Sauvegarder la production brute pour la Suisse (CH)
production_CH = raw_productions_by_src['CH']['raw_df']

# Sauvegarder la production brute pour la France (FR)
production_FR = raw_productions_by_src['FR']['raw_df']


# Sauvegarder la production brute pour l'Allemagne (DE)
production_DE = raw_productions_by_src['DE']['raw_df']

In [18]:
production_CH['Hydro_Pumped_Storage_CH'].loc['2023'].sum()

np.float64(11013203.05873742)

In [19]:
tot_electricity_mix_IT = electricity_mixs['IT']['df']


# Sauvegarder le mix électrique total pour l'Autriche (AT)
tot_electricity_mix_AT = electricity_mixs['AT']['df']


# Sauvegarder le mix électrique total pour la Suisse (CH)
tot_electricity_mix_CH = electricity_mixs['CH']['df']


# Sauvegarder le mix électrique total pour la France (FR)
tot_electricity_mix_FR = electricity_mixs['FR']['df']


# Sauvegarder le mix électrique total pour l'Allemagne (DE)
tot_electricity_mix_DE = electricity_mixs['DE']['df']

In [20]:
tot_electricity_mix_DE

,AT,CH,DE,FR,IT,Other,sum
2016-01-01 00:00:00,0.000047,0.000818,0.962715,0.010885,1.879888e-05,0.0255,0.999984
2016-01-01 01:00:00,0.000029,0.000485,0.961875,0.011338,8.932580e-06,0.0259,0.999636
2016-01-01 02:00:00,0.000014,0.000193,0.966517,0.007248,1.975910e-06,0.0264,1.000375
2016-01-01 03:00:00,0.000024,0.000412,0.957786,0.018313,4.137470e-06,0.0240,1.000538
2016-01-01 04:00:00,0.000035,0.000639,0.954536,0.020435,6.224920e-06,0.0241,0.999750
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.000167,0.000087,0.954058,0.008083,1.562920e-07,0.0371,0.999495
2024-12-31 20:00:00,0.000051,0.000189,0.940604,0.015016,4.420870e-07,0.0436,0.999461
2024-12-31 21:00:00,0.000042,0.000153,0.929953,0.022698,4.679430e-09,0.0466,0.999446
2024-12-31 22:00:00,0.000062,0.000475,0.928327,0.022271,7.849000e-07,0.0489,1.000036


In [21]:
tot_electricity_impact_IT = {}
tot_electricity_impact_AT = {}
tot_electricity_impact_CH = {}
tot_electricity_impact_FR = {}
tot_electricity_impact_DE = {}
for voltage in ['At plant', 'High Voltage','Medium Voltage','Low Voltage']:
    tot_electricity_impact_IT[voltage] = electricity_impacts['IT'][voltage]['df']


    # Sauvegarder l'impact électrique total pour l'Autriche (AT)
    tot_electricity_impact_AT[voltage] = electricity_impacts['AT'][voltage]['df']


    # Sauvegarder l'impact électrique total pour la Suisse (CH)
    tot_electricity_impact_CH[voltage] = electricity_impacts['CH'][voltage]['df']


    # Sauvegarder l'impact électrique total pour la France (FR)
    tot_electricity_impact_FR[voltage] = electricity_impacts['FR'][voltage]['df']


    # Sauvegarder l'impact électrique total pour l'Allemagne (DE)
    tot_electricity_impact_DE[voltage] = electricity_impacts['DE'][voltage]['df']

In [22]:
tot_electricity_impact_DE

{'At plant':                            AT        CH          DE        FR        IT  \
 2016-01-01 00:00:00  0.012704  0.010915  516.529197  0.388215  0.019250   
 2016-01-01 01:00:00  0.008008  0.006280  515.950693  0.412417  0.009130   
 2016-01-01 02:00:00  0.003710  0.002499  517.592229  0.269714  0.002019   
 2016-01-01 03:00:00  0.006380  0.005341  514.726980  0.701124  0.004224   
 2016-01-01 04:00:00  0.008991  0.008243  527.298301  0.760291  0.006333   
 ...                       ...       ...         ...       ...       ...   
 2024-12-31 19:00:00  0.052296  0.001294  299.646270  0.201038  0.000102   
 2024-12-31 20:00:00  0.015695  0.002599  258.930150  0.350773  0.000289   
 2024-12-31 21:00:00  0.012768  0.002074  254.786710  0.553159  0.000003   
 2024-12-31 22:00:00  0.019405  0.005805  226.065320  0.564645  0.000513   
 2024-12-31 23:00:00  0.049990  0.001241  217.197850  0.372888  0.000984   
 
                      Other         sum  
 2016-01-01 00:00:00   1.14  518

In [23]:
src_mapping = {'Mix_Other' : 'Other',
'Residual_Hydro_Run-of-river_and_poundage': 'Hydro run-of-river',
'Biomass' :'Biomass' ,
'Fossil_Coal-derived_gas': 'Gas',
'Fossil_Oil':'Oil',
'Geothermal' :'Geothermal',
'Hydro_Run-of-river_and_poundage' :'Hydro run-of-river',
'Marine': 'Other',
'Solar' : 'Solar',
'Residual_Hydro_Water_Reservoir' : 'Hydro lake',
'Residual_Other': 'Other',
'Fossil_Brown_coal/Lignite':'Coal',
'Fossil_Gas': 'Gas',
'Fossil_Oil_shale':'Oil',
'Hydro_Pumped_Storage' : 'Hydro storage',
'Hydro_Water_Reservoir' : 'Hydro lake',
'Nuclear' : 'Nuclear',
'Waste' : 'Waste',
'Fossil_Hard_coal':'Coal',
'Fossil_Peat':'Coal',
'Other_fossil': 'Other',
'Wind_Offshore' : 'Wind',
'Other_renewable': 'Other',
'Wind_Onshore' : 'Wind'}


def group_by_src(DF):
    # Supposons que df soit votre DataFrame
    df = DF.copy()
    df.columns = [col[:-3] if col != 'Mix_Other' else col for col in df.columns]
    df = df.groupby(df.columns, axis=1).sum()
    df = pd.concat([df, df[['Mix_Other']]], axis=1)
    df = df.rename(columns=src_mapping)
    df = df.groupby(df.columns, axis=1).sum()

    return df

In [24]:
Techno_impact_IT = {}
Techno_impact_AT = {}
Techno_impact_FR = {}
Techno_impact_DE = {}
Techno_impact_CH = {}
for voltage in ['At plant', 'High Voltage','Medium Voltage','Low Voltage']:

    Techno_impact_IT[voltage] = group_by_src(electricity_impact_by_src_IT[voltage])


    Techno_impact_AT[voltage] = group_by_src(electricity_impact_by_src_AT[voltage])


    Techno_impact_CH[voltage] = group_by_src(electricity_impact_by_src_CH[voltage])

    Techno_impact_FR[voltage] = group_by_src(electricity_impact_by_src_FR[voltage])


    Techno_impact_DE[voltage] = group_by_src(electricity_impact_by_src_DE[voltage])

# Pour l'Italie (IT)
Techno_IT = group_by_src(raw_consumption_by_src_IT)


Techno_AT = group_by_src(raw_consumption_by_src_AT)


Techno_CH = group_by_src(raw_consumption_by_src_CH)


Techno_FR = group_by_src(raw_consumption_by_src_FR)


Techno_DE = group_by_src(raw_consumption_by_src_DE)

C:\Users\PC Jules\AppData\Local\Temp\ipykernel_9120\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_9120\653172070.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_9120\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_9120\653172070.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()
C:\Users\PC Jules\AppData\Local\Temp\ipykernel_9120\653172070.py:31: FutureWarning: DataFrame.groupby with axis=1 is

In [25]:
Techno_IT

,Biomass,Coal,Energy_storage,Gas,Geothermal,Hydro lake,Hydro run-of-river,Hydro storage,Hydro_Pumpage,Nuclear,Oil,Other,Solar,Waste,Wind
2016-01-01 00:00:00,296.811724,1997.960944,0.000000,5335.814164,702.668036,253.809600,1359.646384,68.082340,0.000000,1952.451480,524.449972,10226.401649,1.012074,59.182762,385.359684
2016-01-01 01:00:00,294.666100,1998.366133,0.000000,4800.153167,704.047784,176.195488,1267.933389,35.705476,0.000000,1694.224080,535.189954,9694.139468,0.013122,56.821800,415.680591
2016-01-01 02:00:00,304.892235,2033.540411,0.000000,4582.057670,701.507379,185.136304,1198.714505,35.685605,0.000000,1521.623836,470.754908,9384.629195,0.012565,53.248728,383.664844
2016-01-01 03:00:00,295.225992,1929.509924,0.000000,4228.681650,702.321642,185.814201,1185.328818,39.300534,0.000000,1868.270000,430.862394,8869.843884,0.011269,57.123830,354.819872
2016-01-01 04:00:00,295.291270,1927.555265,0.000000,4263.388465,702.097995,196.245081,1210.205275,44.216802,0.000000,2020.590350,412.871530,8716.275539,0.016344,54.255680,351.727429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,708.131403,560.913980,0.000000,18293.504566,600.487991,796.944783,2838.164748,589.859541,-3.359485,1955.700080,33.246901,2567.714223,0.313452,343.627302,1096.931778
2024-12-31 20:00:00,732.780948,542.972930,0.000000,15740.005657,602.496932,640.699851,2566.880592,331.172457,-3.501286,2103.507220,35.018335,3106.458915,0.298156,355.708815,1295.147722
2024-12-31 21:00:00,742.257401,540.424395,0.000000,13826.501412,603.231344,535.515837,2254.225558,309.409489,-3.660447,2242.024067,36.070659,3467.516455,0.288514,375.450062,1408.161426
2024-12-31 22:00:00,738.460186,509.485600,0.000000,12718.475191,600.935228,599.664551,2313.750470,276.861334,-3.576197,2324.528050,36.717256,3397.984816,0.237107,378.293058,1565.186303


## Sauvegarde des DataFrames au format parquet.gz

In [26]:
base_dir = r"C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK"

folders = {
    'consumptions': {
        'raw_consumption_by_src_AT': raw_consumption_by_src_AT,
        'raw_consumption_by_src_CH': raw_consumption_by_src_CH,
        'raw_consumption_by_src_DE': raw_consumption_by_src_DE,
        'raw_consumption_by_src_FR': raw_consumption_by_src_FR,
        'raw_consumption_by_src_IT': raw_consumption_by_src_IT,
        'tot_consumption_AT': tot_consumption_AT,
        'tot_consumption_CH': tot_consumption_CH,
        'tot_consumption_DE': tot_consumption_DE,
        'tot_consumption_FR': tot_consumption_FR,
        'tot_consumption_IT': tot_consumption_IT
    },
    'electricity_impacts': {
         # --- At plant ---
        'At plant' : {
            'electricity_impact_AT_AP': tot_electricity_impact_AT['At plant'],
            'electricity_impact_CH_AP': tot_electricity_impact_CH['At plant'],
            'electricity_impact_DE_AP': tot_electricity_impact_DE['At plant'],
            'electricity_impact_FR_AP': tot_electricity_impact_FR['At plant'],
            'electricity_impact_IT_AP': tot_electricity_impact_IT['At plant'],
            'electricity_impact_by_src_AT_AP': electricity_impact_by_src_AT['At plant'],
            'electricity_impact_by_src_CH_AP': electricity_impact_by_src_CH['At plant'],
            'electricity_impact_by_src_DE_AP': electricity_impact_by_src_DE['At plant'],
            'electricity_impact_by_src_FR_AP': electricity_impact_by_src_FR['At plant'],
            'electricity_impact_by_src_IT_AP': electricity_impact_by_src_IT['At plant']
        },
        # --- HIGH VOLTAGE ---
        'High Voltage' : {
            'electricity_impact_AT_HV': tot_electricity_impact_AT['High Voltage'],
            'electricity_impact_CH_HV': tot_electricity_impact_CH['High Voltage'],
            'electricity_impact_DE_HV': tot_electricity_impact_DE['High Voltage'],
            'electricity_impact_FR_HV': tot_electricity_impact_FR['High Voltage'],
            'electricity_impact_IT_HV': tot_electricity_impact_IT['High Voltage'],
            'electricity_impact_by_src_AT_HV': electricity_impact_by_src_AT['High Voltage'],
            'electricity_impact_by_src_CH_HV': electricity_impact_by_src_CH['High Voltage'],
            'electricity_impact_by_src_DE_HV': electricity_impact_by_src_DE['High Voltage'],
            'electricity_impact_by_src_FR_HV': electricity_impact_by_src_FR['High Voltage'],
            'electricity_impact_by_src_IT_HV': electricity_impact_by_src_IT['High Voltage']
        },
        # --- MEDIUM VOLTAGE ---
        'Medium Voltage' : {
            'electricity_impact_AT_MV': tot_electricity_impact_AT['Medium Voltage'],
            'electricity_impact_CH_MV': tot_electricity_impact_CH['Medium Voltage'],
            'electricity_impact_DE_MV': tot_electricity_impact_DE['Medium Voltage'],
            'electricity_impact_FR_MV': tot_electricity_impact_FR['Medium Voltage'],
            'electricity_impact_IT_MV': tot_electricity_impact_IT['Medium Voltage'],
            'electricity_impact_by_src_AT_MV': electricity_impact_by_src_AT['Medium Voltage'],
            'electricity_impact_by_src_CH_MV': electricity_impact_by_src_CH['Medium Voltage'],
            'electricity_impact_by_src_DE_MV': electricity_impact_by_src_DE['Medium Voltage'],
            'electricity_impact_by_src_FR_MV': electricity_impact_by_src_FR['Medium Voltage'],
            'electricity_impact_by_src_IT_MV': electricity_impact_by_src_IT['Medium Voltage']
        },
        # --- LOW VOLTAGE ---
        'Low Voltage' : {
            'electricity_impact_AT_LV': tot_electricity_impact_AT['Low Voltage'],
            'electricity_impact_CH_LV': tot_electricity_impact_CH['Low Voltage'],
            'electricity_impact_DE_LV': tot_electricity_impact_DE['Low Voltage'],
            'electricity_impact_FR_LV': tot_electricity_impact_FR['Low Voltage'],
            'electricity_impact_IT_LV': tot_electricity_impact_IT['Low Voltage'],
            'electricity_impact_by_src_AT_LV': electricity_impact_by_src_AT['Low Voltage'],
            'electricity_impact_by_src_CH_LV': electricity_impact_by_src_CH['Low Voltage'],
            'electricity_impact_by_src_DE_LV': electricity_impact_by_src_DE['Low Voltage'],
            'electricity_impact_by_src_FR_LV': electricity_impact_by_src_FR['Low Voltage'],
            'electricity_impact_by_src_IT_LV': electricity_impact_by_src_IT['Low Voltage']
        }
    },
    'electricity_mixs': {
        'electricity_mix_AT': tot_electricity_mix_AT,
        'electricity_mix_CH': tot_electricity_mix_CH,
        'electricity_mix_DE': tot_electricity_mix_DE,
        'electricity_mix_FR': tot_electricity_mix_FR,
        'electricity_mix_IT': tot_electricity_mix_IT
    },
    'flows': {
        'flows_AT': flows_AT,
        'flows_CH': flows_CH,
        'flows_DE': flows_DE,
        'flows_FR': flows_FR,
        'flows_IT': flows_IT
    },
    'productions': {
        'production_AT': production_AT,
        'production_CH': production_CH,
        'production_DE': production_DE,
        'production_FR': production_FR,
        'production_IT': production_IT,
        'tot_production_AT': tot_production_AT,
        'tot_production_CH': tot_production_CH,
        'tot_production_DE': tot_production_DE,
        'tot_production_FR': tot_production_FR,
        'tot_production_IT': tot_production_IT
    },
    'technologies': {
        'technologies_AT': Techno_AT,
        'technologies_CH': Techno_CH,
        'technologies_DE': Techno_DE,
        'technologies_FR': Techno_FR,
        'technologies_IT': Techno_IT,
        # --- At plant ---
        'At plant' : {
            'Techno_impact_AT_AP': Techno_impact_AT['At plant'],
            'Techno_impact_CH_AP': Techno_impact_CH['At plant'],
            'Techno_impact_DE_AP': Techno_impact_DE['At plant'],
            'Techno_impact_FR_AP': Techno_impact_FR['At plant'],
            'Techno_impact_IT_AP': Techno_impact_IT['At plant']
        },
        # --- HIGH VOLTAGE ---
        'High Voltage' : {
            'Techno_impact_AT_HV': Techno_impact_AT['High Voltage'],
            'Techno_impact_CH_HV': Techno_impact_CH['High Voltage'],
            'Techno_impact_DE_HV': Techno_impact_DE['High Voltage'],
            'Techno_impact_FR_HV': Techno_impact_FR['High Voltage'],
            'Techno_impact_IT_HV': Techno_impact_IT['High Voltage']
        },
        # --- MEDIUM VOLTAGE ---
        'Medium Voltage' : {
            'Techno_impact_AT_MV': Techno_impact_AT['Medium Voltage'],
            'Techno_impact_CH_MV': Techno_impact_CH['Medium Voltage'],
            'Techno_impact_DE_MV': Techno_impact_DE['Medium Voltage'],
            'Techno_impact_FR_MV': Techno_impact_FR['Medium Voltage'],
            'Techno_impact_IT_MV': Techno_impact_IT['Medium Voltage']
        },
        # --- LOW VOLTAGE ---
        'Low Voltage' : {
            'Techno_impact_AT_LV': Techno_impact_AT['Low Voltage'],
            'Techno_impact_CH_LV': Techno_impact_CH['Low Voltage'],
            'Techno_impact_DE_LV': Techno_impact_DE['Low Voltage'],
            'Techno_impact_FR_LV': Techno_impact_FR['Low Voltage'],
            'Techno_impact_IT_LV': Techno_impact_IT['Low Voltage']
        }
    }
}

for folder_name, dataframes in folders.items():
    folder_path = os.path.join(base_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    for file_name, df in dataframes.items():
        if isinstance(df, dict):
            sub_folder_name = file_name
            sub_folder_content = df
            # Creation of the subfolder for voltage level
            sub_folder_path = os.path.join(folder_path, sub_folder_name)
            os.makedirs(sub_folder_path, exist_ok=True)
            for sub_file_name, sub_df in sub_folder_content.items():
                file_path = os.path.join(sub_folder_path, f'{sub_file_name}.parquet.gz')
                sub_df.to_parquet(file_path, compression='gzip')
                print(f'Saved (Sub-folder): {file_path}')
        elif isinstance(df, pd.DataFrame):
            file_path = os.path.join(folder_path, f'{file_name}.parquet.gz')
            df.to_parquet(file_path, compression='gzip')
            print(f'Saved (Direct): {file_path}')

print('\nToutes les données ont été sauvegardées avec succès!')

Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\raw_consumption_by_src_AT.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\raw_consumption_by_src_CH.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\raw_consumption_by_src_DE.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\raw_consumption_by_src_FR.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\raw_consumption_by_src_IT.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\tot_consumption_AT.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\tot_consumption_CH.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\tot_consumption_DE.parquet.gz
Saved (Direct): C:\Users\PC Jules\Desktop\EcoDynElec_data\UVEK\consumptions\tot_consumption_FR.parquet.gz
Saved (Dire